In [3]:
!pip install scipy shapely alphashape

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.2/535.2 kB 7.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.7/694.7 kB 19.5 MB/s eta 0:00:0000:01


In [11]:
import open3d as o3d
import numpy as np
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.spatial import ConvexHull
from sklearn.cluster import DBSCAN
from sklearn.linear_model import RANSACRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

from scipy.spatial import Delaunay
from shapely.geometry import MultiPoint, Polygon
import alphashape
from scipy.optimize import least_squares
from scipy import stats
import os, re

In [12]:
def extract_integers_from_filenames(directory):
    # 正規表現パターンを定義
    pattern = re.compile(r'^(\d+)\.pcd$')
    
    # 指定ディレクトリ内のファイルを取得
    files = os.listdir(directory)
    
    # 整数を格納するセット（重複を避けるため）
    integers = set()
    
    # 各ファイルをチェック
    for file in files:
        match = pattern.match(file)
        if match:
            # ファイル名から整数を抽出してセットに追加
            integer = int(match.group(1))
            integers.add(integer)
    
    # 結果をソートしてリストとして返す
    return sorted(integers)



In [13]:
pcd_g  = o3d.io.read_point_cloud(
    "/home/aichi2204/Documents/bkl2go/20240412-library2/minimini_1/sp_25/ground_merged.pcd"
    )


In [14]:
# 使用例
directory = '/home/aichi2204/Documents/bkl2go/20240412-library2/minimini_1/sp_25/wall_merged'
integers = extract_integers_from_filenames(directory)

for l in integers:  
    pcd_w  = o3d.io.read_point_cloud(
        "/home/aichi2204/Documents/bkl2go/20240412-library2/minimini_1/sp_25/wall_merged/{}.pcd".format(l)
        )
    pcd_w_tree = o3d.geometry.KDTreeFlann(pcd_w)
    edges = []
    for p in pcd_g.points:
        k, neighbor_idx, _ = pcd_w_tree.search_hybrid_vector_3d(
            p ,0.02, 1)
        if k >= 1:
            edges.append(p)
    if len(edges) <2:
        continue
    edges = np.array(edges)
    pcd = o3d.geometry.PointCloud() 
    pcd.points = o3d.utility.Vector3dVector(edges)
    o3d.io.write_point_cloud("/home/aichi2204/Documents/bkl2go/20240412-library2/minimini_1/sp_25/wall_merged/{}_edge.ply".format(l), pcd)

    # 3次元データの例
    X = edges

    # 平均を引く
    mean = np.mean(X, axis=0)
    X_c = X - mean

    # 共分散行列を計算
    C = np.cov(X_c.T)

    # 固有値と固有ベクトルを計算
    eigenvalues, eigenvectors = np.linalg.eig(C)

    # 最大の固有値に対応する固有ベクトル
    principal_component = eigenvectors[:, np.argmax(eigenvalues)]

    # 直線の方程式
    point_on_line = mean
    direction = principal_component

    # 各点を直線に射影する関数
    def project_point_onto_line(point, line_point, line_dir):
        return line_point + np.dot(point - line_point, line_dir) * line_dir

    # 各点の射影位置を計算
    projected_points = np.array([project_point_onto_line(p, point_on_line, direction) for p in X])

    d_dot_d = np.dot(direction, direction)

    def calculate_t(q, p, d):
        return np.dot(q - p, d) / d_dot_d

    t_values = np.array([calculate_t(q, point_on_line, direction) for q in projected_points])
    point_far_1= projected_points[np.argmin(t_values)]
    point_far_2 = projected_points[np.argmax(t_values)]

    print(l, point_far_1,point_far_2)

    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(np.array([point_far_1,point_far_2]))
    line_set.lines = o3d.utility.Vector2iVector(np.array([[0,1]]))
    line_set.paint_uniform_color([1, 0, 0])
    o3d.io.write_line_set("/home/aichi2204/Documents/bkl2go/20240412-library2/minimini_1/sp_25/wall_merged/{}_line.ply".format(l), 
                          line_set)
    # o3d.visualization.draw_geometries([line_set,pcd_w])

15 [-64.59694917   6.05371255   0.92174578] [-62.27987087   3.24585958   0.75972674]
16 [-65.39161312   6.9910512    0.98855817] [-65.45781015   7.07797958   0.98243883]
17 [-66.15312822   7.92048793   1.02650075] [-66.29532131   8.09552467   1.0374417 ]
19 [-67.38327764   9.38485395   1.07567758] [-67.17799349   9.18316275   1.06804764]
20 [-67.41475097  14.15107919   1.08799905] [-68.26587973  15.17847604   1.07536416]
26 [-65.22679016  11.49205997   1.13305541] [-65.73372679  11.06675364   1.12008051]
27 [-66.61265943   8.48225703   1.06130984] [-66.88398259   8.79838118   1.06846311]
31 [-66.43309265   7.94449925   0.34296668] [-67.3352299    9.03345535   0.33093214]
32 [-66.49051123   8.3266599    1.04986791] [-66.46995957   8.319289     1.05064256]
42 [-71.561673    12.79490766   0.97453913] [-69.80379671  14.27745758   1.02075379]
48 [-67.2606203   14.04348454   1.08858961] [-66.58971869  13.2097487    1.10136984]
49 [-67.39520232   9.07862637   0.32538057] [-68.96382242   7.780

75, 185, [-72.05469906  12.39532089   0.37688806] [-68.66289396   8.29641541   0.43524847]

75, 61,  [-72.27247327  12.21088763   0.36600007] [-72.08606566  12.3553822    0.37108409]

75, 130, [-68.87907923   8.13774736   0.42486485] [-68.68904212   8.28927063   0.43933543]

75, 132, [-72.26375762  12.21722198   0.37063058] [-68.9091871    8.14556969   0.42639707]

73
2 [-69.77962737  10.10490788   0.80267984] [-69.84932937  10.23611661   0.80835588]

42 [-71.74696955  12.65995692   0.76991916] [-71.58811889  12.78161598   0.78529111]

110 [-68.25067965   8.73830064   0.84316038] [-69.35938232  10.07883337   0.82240456]

118 [-70.77448274  11.36446183   0.79279789] [-71.7608559   12.54038849   0.77181453]

330 [-71.59733861  12.77940178   0.78410218] [-71.091798    12.16301695   0.7906985 ]

347 [-68.43590112   8.51478578   0.83452875] [-68.22480507   8.6717018    0.84130771]

836 [-69.82937776  10.64806923   0.81255395] [-69.33970012  10.05327296   0.82184913]

905 [-70.7653761   11.34448791   0.7928443 ] [-68.44336159   8.5347394    0.83097529]

1041 [-69.85341181  10.6735566    0.80966516] [-71.04370133  12.11386012   0.7890005 ]

1511 [-71.15334216  11.81172778   0.78401768] [-71.52307914  12.25388424   0.7745863 ]

50
42 [-72.03459241  12.37175497   0.56353611] [-71.83075934  12.58874586   0.57801318] 

185 [-72.03088331  12.40253394   0.57088157] [-68.74108561   8.4245429    0.62302238]

318 [-71.82830697  12.58012385   0.57604733] [-70.35449093  10.8150416    0.60461236]

324 [-70.33977448  10.79431644   0.60289425] [-68.49804696   8.5634516    0.63618582]

622 [-68.64874575   8.32316134   0.63421174] [-68.44059955   8.49870398   0.63255699]

In [89]:
# 例として，簡単なLineSetを作成
points = np.array([[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0]])
lines = np.array([[0, 1], [1, 2], [2, 3], [3, 0]])

# LineSetオブジェクトを作成
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(points)
line_set.lines = o3d.utility.Vector2iVector(lines)

# 保存
o3d.io.write_line_set("lineset.ply", line_set)

True